# 🗡️ Dark Fantasy Lore Generator - Local Model Edition

Este notebook ejecuta el generador de lore usando un modelo descargado de HuggingFace, evitando problemas de quota de API.

**Requisitos:**
- GPU Runtime (Runtime → Change runtime type → T4 GPU)
- Token de HuggingFace (para modelos con licencia)

---

## 📦 1. Instalar Dependencias

In [ ]:
!pip install -q transformers accelerate bitsandbytes torch
!pip install -q huggingface_hub sentencepiece pyyaml

print("✓ Dependencias instaladas")

## 🔑 2. Login en HuggingFace (Opcional)

Solo necesario para modelos con licencia (Llama, Mistral, etc.)

Obtené tu token en: https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import login

# Descomenta la siguiente línea y pega tu token
# login(token="hf_xxxxxxxxxxxxxxxxxxxx")

# O usa el login interactivo:
login()

## 🤖 3. Cargar Modelo

Opciones de modelos (descomentar la que prefieras):

| Modelo | VRAM | Calidad |
|--------|------|---------|
| Mistral-7B-Instruct | ~8GB | ⭐⭐⭐⭐ |
| Qwen2.5-7B-Instruct | ~8GB | ⭐⭐⭐⭐ |
| SmolLM2-1.7B-Instruct | ~4GB | ⭐⭐⭐ (Colab Free) |

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# === ELEGIR MODELO ===
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
# model_id = "Qwen/Qwen2.5-7B-Instruct"
# model_id = "HuggingFaceTB/SmolLM2-1.7B-Instruct"  # Para Colab Free

print(f"Cargando modelo: {model_id}")
print("Esto puede tomar 5-10 minutos...")

# Quantización 4-bit para ahorrar VRAM
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Verificar VRAM usada
!nvidia-smi --query-gpu=memory.used --format=csv
print("\n✓ Modelo cargado exitosamente!")

## 🔧 4. Crear LLMService Local

In [ ]:
import json
import re
from typing import Optional

class LocalLLMService:
    """
    Servicio LLM local compatible con la estructura del proyecto.
    Reemplaza las llamadas a Gemini API.
    """

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.total_tokens_generated = 0

    def count_tokens(self, text: str) -> int:
        """Contar tokens en un texto."""
        return len(self.tokenizer.encode(text))

    def generate_content(
        self,
        prompt: str,
        generation_config: dict = None,
        retries: int = 3,
        caller: str = "unknown"
    ) -> Optional[str]:
        """
        Generar contenido usando el modelo local.
        Interfaz compatible con LLMService original.
        """
        print(f"\n[{caller}] Generando respuesta...")
        input_tokens = self.count_tokens(prompt)
        print(f"[{caller}] Input tokens: {input_tokens}")

        # Preparar prompt en formato chat
        messages = [{"role": "user", "content": prompt}]

        try:
            # Aplicar template del modelo
            if hasattr(self.tokenizer, 'apply_chat_template'):
                input_text = self.tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True
                )
            else:
                # Fallback para modelos sin chat template
                input_text = f"[INST] {prompt} [/INST]"

            inputs = self.tokenizer(input_text, return_tensors="pt").to("cuda")

            # Generar
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=4096,
                    temperature=0.85,
                    top_p=0.95,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.1
                )

            # Decodificar respuesta
            full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extraer solo la respuesta del assistant
            if "[/INST]" in full_response:
                response = full_response.split("[/INST]")[-1].strip()
            elif "assistant" in full_response.lower():
                response = full_response.split("assistant")[-1].strip()
            else:
                # Remover el prompt del inicio
                response = full_response[len(input_text):].strip()

            output_tokens = self.count_tokens(response)
            self.total_tokens_generated += output_tokens

            print(f"[{caller}] Output tokens: {output_tokens}")
            print(f"[{caller}] ✓ Generación completada")

            return response

        except Exception as e:
            print(f"[{caller}] ✗ Error: {e}")
            return None

# Instanciar servicio
llm_service = LocalLLMService(model, tokenizer)
print("✓ LocalLLMService creado")

## 📥 5. Descargar Código del Proyecto

In [ ]:
# Opción A: Clonar desde GitHub (si lo tenés subido)
# !git clone https://github.com/TU_USUARIO/dark-fantasy-book-generator.git

# Opción B: Crear estructura mínima localmente
import os

os.makedirs("backend/agents", exist_ok=True)
os.makedirs("backend/services", exist_ok=True)
os.makedirs("backend/data", exist_ok=True)

print("✓ Estructura de directorios creada")
print("\n📁 Ahora sube los archivos del backend usando el panel de archivos de Colab")
print("   o usa la siguiente celda para subir un .zip")

In [ ]:
# Opción C: Subir archivo .zip con el backend
from google.colab import files
import zipfile

print("Selecciona el archivo .zip con la carpeta 'backend'")
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"✓ {filename} extraído")

## 🔌 6. Configurar Agentes con LLM Local

In [ ]:
import sys
sys.path.insert(0, '/content')

# Importar componentes
from backend.agents.lore_state_manager import LoreStateManager
from backend.agents.era_architect import EraArchitectAgent
from backend.agents.faction_forge import FactionForgeAgent
from backend.agents.soul_weaver import SoulWeaverAgent
from backend.agents.conflict_designer import ConflictDesignerAgent
from backend.agents.pathweaver import PathweaverAgent
from backend.services.variety_injector import VarietyInjector

print("✓ Módulos importados")

# Crear instancias con nuestro LLM local
state_manager = LoreStateManager()
variety_injector = VarietyInjector()

era_agent = EraArchitectAgent(llm_service, state_manager)
faction_agent = FactionForgeAgent(llm_service, state_manager)
soul_agent = SoulWeaverAgent(llm_service, state_manager)
conflict_agent = ConflictDesignerAgent(llm_service, state_manager)
path_agent = PathweaverAgent(llm_service, state_manager)

print("✓ Agentes configurados con LLM local")

## 🗡️ 7. Generar Lore!

In [ ]:
# === CONFIGURACIÓN DEL PROYECTO ===
PROJECT_NAME = "Chronicles of the Shattered Crown"
NUM_ERAS = 3
NUM_FACTIONS = 4
NUM_CHARACTERS = 5
NUM_CONFLICTS = 3
NUM_CHAPTERS_PER_ROUTE = 3

print(f"\n{'='*50}")
print(f"🗡️ GENERANDO LORE: {PROJECT_NAME}")
print(f"{'='*50}\n")

# Obtener seeds de variedad
variety_seeds = variety_injector.get_generation_seeds()
print(f"Culturas: {variety_seeds['name_cultures']}")
print(f"Emoción: {variety_seeds['emotion_seed']}")
print(f"Estética: {variety_seeds['aesthetic_seed']}")

# Inicializar state
state_manager.set_project_info(PROJECT_NAME, "dark_fantasy")
state_manager.set_variety_seeds(variety_seeds)

In [ ]:
# Fase 1: Generar Eras
print("\n📜 FASE 1: Generando Eras...")
eras_result = era_agent.process(PROJECT_NAME, variety_seeds, num_eras=NUM_ERAS)

if eras_result.get('eras'):
    print(f"\n✓ {len(eras_result['eras'])} eras generadas:")
    for era in eras_result['eras']:
        print(f"  - {era.get('name', 'Unknown')}")
else:
    print("✗ Error generando eras")

In [ ]:
# Fase 2: Generar Facciones
print("\n⚔️ FASE 2: Generando Facciones...")
factions_result = faction_agent.process(
    PROJECT_NAME,
    variety_seeds,
    eras_result.get('eras', []),
    num_factions=NUM_FACTIONS
)

if factions_result.get('factions'):
    print(f"\n✓ {len(factions_result['factions'])} facciones generadas:")
    for faction in factions_result['factions']:
        print(f"  - {faction.get('name', 'Unknown')}")
else:
    print("✗ Error generando facciones")

In [ ]:
# Fase 3: Generar Personajes
print("\n👤 FASE 3: Generando Personajes...")
characters_result = soul_agent.process(
    PROJECT_NAME,
    variety_seeds,
    factions_result.get('factions', []),
    num_characters=NUM_CHARACTERS
)

characters = characters_result.get('characters', [])
if characters:
    print(f"\n✓ {len(characters)} personajes generados:")
    for char in characters:
        print(f"  - {char.get('name', 'Unknown')} ({char.get('archetype', 'Unknown')})")
else:
    print("✗ Error generando personajes")

In [ ]:
# Fase 4: Generar Conflictos
print("\n🔥 FASE 4: Generando Conflictos...")
conflicts_result = conflict_agent.process(
    PROJECT_NAME,
    variety_seeds,
    factions_result.get('factions', []),
    characters,
    num_conflicts=NUM_CONFLICTS
)

conflicts = conflicts_result.get('conflicts', [])
if conflicts:
    print(f"\n✓ {len(conflicts)} conflictos generados:")
    for conflict in conflicts:
        print(f"  - {conflict.get('name', 'Unknown')}")
else:
    print("✗ Error generando conflictos")

In [ ]:
# Fase 5: Generar Rutas
print("\n📖 FASE 5: Generando Rutas Narrativas...")
routes_result = path_agent.process(
    PROJECT_NAME,
    variety_seeds,
    factions_result.get('factions', []),
    characters,
    conflicts,
    num_chapters_per_route=NUM_CHAPTERS_PER_ROUTE
)

routes = routes_result.get('routes', {})
if routes:
    print(f"\n✓ Rutas generadas:")
    for route_name, route_data in routes.items():
        chapters = route_data.get('chapters', [])
        print(f"  - {route_name}: {len(chapters)} capítulos")
else:
    print("✗ Error generando rutas")

## 💾 8. Guardar Resultados

In [ ]:
# Obtener estado final
final_state = state_manager.get_state()

# Guardar como JSON
output_filename = PROJECT_NAME.replace(' ', '_') + '_lore.json'
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(final_state, f, indent=2, ensure_ascii=False)

print(f"\n✓ Lore guardado en: {output_filename}")

# Mostrar estadísticas
print(f"\n{'='*50}")
print("📊 ESTADÍSTICAS FINALES")
print(f"{'='*50}")
print(f"Tokens generados: {llm_service.total_tokens_generated:,}")
print(f"Eras: {len(final_state.get('eras', []))}")
print(f"Facciones: {len(final_state.get('factions', []))}")
print(f"Personajes: {len(final_state.get('characters', []))}")
print(f"Conflictos: {len(final_state.get('conflicts', []))}")

In [ ]:
# Descargar archivo
from google.colab import files
files.download(output_filename)

## 🔍 9. Preview del Lore Generado

In [ ]:
# Mostrar preview del lore
from IPython.display import display, Markdown

state = state_manager.get_state()

md_output = f"# {state.get('project_name', 'Lore')}\n\n"

# Cosmology
if state.get('cosmology'):
    cosmo = state['cosmology']
    md_output += "## 🌌 Cosmology\n\n"
    md_output += f"**Creation Myth:** {cosmo.get('creation_myth', 'N/A')}\n\n"
    md_output += f"**Divine Forces:** {cosmo.get('divine_forces', 'N/A')}\n\n"

# Eras
if state.get('eras'):
    md_output += "## ⏳ Eras\n\n"
    for era in state['eras']:
        md_output += f"### {era.get('name', 'Unknown Era')}\n"
        md_output += f"{era.get('summary', '')}\n\n"

# Factions
if state.get('factions'):
    md_output += "## ⚔️ Factions\n\n"
    for faction in state['factions']:
        md_output += f"### {faction.get('name', 'Unknown')}\n"
        md_output += f"{faction.get('description', faction.get('summary', ''))}\n\n"

# Characters
if state.get('characters'):
    md_output += "## 👤 Characters\n\n"
    for char in state['characters']:
        md_output += f"### {char.get('name', 'Unknown')} ({char.get('archetype', 'Unknown')})\n"
        md_output += f"{char.get('summary', char.get('background', ''))}\n\n"

display(Markdown(md_output[:5000]))  # Limitar preview